<a href="https://colab.research.google.com/github/Reinaldo-34/sentimentGoogle/blob/main/googleReadV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1_vROEMAQ29haCkI1yowbZh1h5qJfFWctnOqhxHneytE/edit#gid=0')

import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sheet = wb.worksheet('Sheet1')
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]

In [ ]:
reviewList = list(df['Review text'])


In [70]:
print(reviewList[1])
print(range(len(reviewList)))

I only have the highest praise for Emmanuel from St Trinity Property Group. His level of professionalism and client service is second to none! I first met Emmanuel with a very clear view of the apartments I wanted to inspect for purchase consideration. However, Emmanuel still took the time to understand my needs and then suggested 3 other apartments I should also look at, in addition to the apartments I initially wanted to view.  After viewing all 5 apartments in June, I proceeded to purchase one of the other apartments Emmanuel had suggested; which was actually slightly cheaper and a far better fit for my needs. When it came to contract exchange I wanted a slight contract variation which was initially declined, yet with Emmanuel’s timely assistance was ultimately agreed to. This clearly evidences the fact Emmanuel always has his client’s best interests in mind. I was extremely lucky to have come across Emmanuel and it is without any hesitation that I would recommend Emmanuel to any pr

In [51]:
## iterate through all items in ReviewList and tokenize
tokens = []

for i in range(len(reviewList)):
  x = nltk.word_tokenize(reviewList[i])
  tokens.append(x)



In [ ]:
pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
import numpy as np

In [120]:
score = np.transpose([]) 

analyzer = SentimentIntensityAnalyzer()
for sentence in reviewList:
  vs = analyzer.polarity_scores(sentence)
  score = np.append(score,vs['compound'])
##print("{:-<65} {}".format(sentence, str(vs)))

df['Score'] = score
 

In [ ]:
print(score)
score.size

[ 0.9598  0.981   0.9856  0.5413  0.854  -0.9044  0.7712  0.8622  0.9611
  0.9516  0.7841  0.8955  0.9782  0.4408  0.4927  0.9523  0.9741  0.659
  0.9488  0.8261  0.9439  0.4025  0.7351  0.9694  0.9677  0.8221  0.8817
  0.9775  0.2152  0.9866  0.9022  0.9035  0.9966  0.8797  0.9416  0.9183
 -0.3766  0.9501  0.9022  0.9651  0.836   0.9571 -0.9514  0.6124  0.984
  0.8999  0.886   0.9825  0.946   0.9413  0.9846  0.9545  0.9022 -0.9125
  0.862   0.6778  0.8517 -0.2263  0.0258  0.4754  0.5719  0.6705  0.8221]


63

In [127]:
## classifying compound scores into positive, negative or neutral
def get_class(score):
  if score >= 0.5:
    return 'Positive'
  elif score <= -0.5:
    return 'Negative'
  else:
    return 'Neutral'

## transforming
df['Sentiment'] = df['Score'].apply(get_class)

In [107]:
consultant = ['Emmanuel', 'Jack', 'Kevin', 'Lisa', 'Mohit', 'Rebekah', 'Reneh', 'Rossetta', 'Sophie', 'Terry']


In [116]:
owner = []
for k in range(63):
  x = 0
  for i in tokens[k]:
    for j in consultant:
      if i == j:
        x = x+1 
        y = j
  if x > 0:
    owner.append(y)
  else:
    owner.append("STPG")

df['Owner'] = owner

In [ ]:
print(owner)

In [130]:
newest_df = {
   ## 'Review Text' : df['Review text'].astype(str),
    'Rating' : df['Rating'].astype(int),
    'Score' : df['Score'].astype(float),
    'Owner' : df['Owner'].astype(str),
    'Sentiment' : df['Sentiment'].astype(str)
}

newDataFrame = pd.DataFrame(newest_df)

In [131]:
print(newDataFrame)

    Rating   Score     Owner Sentiment
1        5  0.9598  Emmanuel  Positive
2        5  0.9810  Emmanuel  Positive
3        5  0.9856      STPG  Positive
4        5  0.5413  Emmanuel  Positive
5        5  0.8540  Emmanuel  Positive
..     ...     ...       ...       ...
59       1  0.0258      STPG   Neutral
60       5  0.4754      STPG   Neutral
61       5  0.5719      STPG  Positive
62       5  0.6705  Emmanuel  Positive
63       5  0.8221      STPG  Positive

[63 rows x 4 columns]
